In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import blimpy as bl 
import setigen as stg 
from astropy import units as u
from astropy.coordinates import Angle
import time

%matplotlib inline

In [2]:
### Load in single-channel Voyager 1 data to compute some GBT noise statistics.
### If you're using a different telescope, you may wish to load a different HDF5 file for waterfall_fn.

waterfall_fn = '/datax/scratch/jaym/single_coarse_guppi_59046_80036_DIAG_VOYAGER-1_0011.rawspec.0000.h5'

fb = bl.Waterfall(waterfall_fn)
print(fb.header)

### Perform a sigma-clipping routine to remove the Voyager signal and most of the PFB rolloff.
### (See C. Choza et al 2024 for justification.)
from astropy.stats import sigma_clip
clipped_data = sigma_clip(fb.data,
                                  sigma=3,
                                  maxiters=5,
                                  masked=False)
### Compute the noise floor and rms. We will use the noise floor to initialize our synthetic frame below.
noise_mean = np.mean(clipped_data)
noise_std = np.std(clipped_data)
print(noise_mean)
print(noise_std)
print(noise_std/noise_mean/np.sqrt(1465))

{'DIMENSION_LABELS': array(['time', 'feed_id', 'frequency'], dtype=object), 'az_start': np.float64(0.0), 'data_type': np.int64(1), 'fch1': np.float64(8421.38671875), 'foff': np.float64(-2.7939677238464355e-06), 'machine_id': np.int64(20), 'nbits': np.int64(32), 'nchans': np.int64(1048576), 'nifs': np.int64(1), 'source_name': 'VOYAGER-1', 'src_dej': <Angle 12.40378167 deg>, 'src_raj': <Angle 17.21124472 hourangle>, 'telescope_id': np.int64(6), 'tsamp': np.float64(18.253611007999982), 'tstart': np.float64(59046.92634259259), 'za_start': np.float64(0.0)}
5.1778175e+06
881581.25
0.004448327710295717


In [3]:
### We need our synthetic data to be readable by turboSETI/BLISS, so we will make it into an HDF5 file with BL's usual header style.
### Most of this information is a placeholder with the precise value being unimportant.

head = {'DIMENSION_LABELS': np.array([b'time', b'feed_id', b'frequency'], dtype=object), 
        'az_start': 0.0, 
        'data_type': 1, 
        'fch1': 6095.214842353016, # This is the edge of a C-band node; it can be changed, but other values below will then also have to be changed.
        'foff': -2.7939677238464355e-06, # Fine-channel frequency resolution.
        'machine_id': 20, 
        'nbits': 32, 
        'nchans': 1048576*16, # 16 coarse channels of 2e20 fine channels each.
        'nifs': 1, 
        'source_name': 'synthetic', # Placeholder name. Some Blimpy functions care about the source name.
        'src_dej': Angle(0*u.deg), # Placeholder declination.
        'src_raj': Angle('0h0m0s'), # Placeholder RA.
        'telescope_id': 6, 
        'tsamp': 18.253611008, # Time resolution.
        'tstart': 60000, # Arbitrary epoch of observation.
        'za_start': 0.0}

In [5]:
### In this cell, we generate 16 coarse channels' worth of chi2 noise using the noise mean we computed from the real GBT data above.

### Read in the polyphase filterbank (PFB) shape. 
### If you're using a different telescope, you can generate a custom PFB shape using BLISS.
### Type the following command in the terminal on the Berkeley Data Center to see how:
### bliss_generate_channelizer_response -h
long_data = []
pfb = np.fromfile('/mnt/blpc2/datax/scratch/benjb/bliss_LSCX_test/0_0_GBT_channelizer_response.f32', dtype='float32')

### Do one coarse channel at a time.
### Setigen frame construction has a nonlinear complexity, so this is faster than doing a single frame with 16 channels.
for i in range(16):

    print(i)

    fchans = (2**20)*1
    tchans = 16
    df = 2.7939677238464355*u.Hz
    dt = 18.253611008*u.s
    fch1 = 6095.214842353016*u.MHz

    frame = stg.Frame(fchans=fchans,
                    tchans=tchans,
                    df=df,
                    dt=dt,
                    fch1=fch1)
    noise = frame.add_noise(x_mean=noise_mean)

    
    pfb_data = np.array([spec*pfb for spec in frame.data])

    long_data.append(pfb_data)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [6]:
### Stack your 16 coarse channels together into a single array.

concat_long_data = np.hstack(long_data)

In [8]:
### Save your 16 coarse channels of noise as a numpy array. 
### (Replace /datax/scratch/benjb/... with your own scratch directory and filepath.)
### This step is optional -- below we'll also save the array as an HDF5 file.

np.save('/datax/scratch/jaym/synthetic_chi2_pfb_data_16ch.npy', concat_long_data)

In [9]:
### Convert your 16-channel numpy array back to a setigen Frame.
### Remember to use the metadata we defined earlier!

frame = stg.Frame.from_data(df = 2.7939677238464355*u.Hz,
                            dt = 18.253611008*u.s,
                            fch1=6095.214842353016*u.MHz,
                            ascending=False,
                            data=concat_long_data,
                            metadata=head)

In [10]:
### Save the 16 channels of noise as an HDF5 file.
### (Replace /datax/scratch/benjb/... with your own scratch directory and filepath.)

frame.save_h5('/datax/scratch/jaym/synthetic_chi2_pfb_data_16ch.h5')

blimpy.waterfall INFO     __write_to_hdf5_light: Writing the spectra matrix for /datax/scratch/jaym/synthetic_chi2_pfb_data_16ch.h5 without blobbing.
hdf5plugin._filters WARNING  Deprecation: hdf5plugin.Bitshuffle's lz4 argument is deprecated, use cname='lz4' or 'none' instead.
hdf5plugin._filters WARNING  Deprecation: hdf5plugin.Bitshuffle's lz4 argument is deprecated, use cname='lz4' or 'none' instead.


blimpy.waterfall INFO     Conversion time: 1.92sec


In [11]:
### Read the noise file back in as a Waterfall object to check that everything looks right.

fb = bl.Waterfall('/datax/scratch/jaym/synthetic_chi2_pfb_data_16ch.h5')
fb.info()


--- File Info ---
DIMENSION_LABELS :   ['time' 'feed_id' 'frequency']
        az_start :                              0.0
       data_type :                                1
            fch1 :            6095.214842353015 MHz
            foff :      -2.7939677238464355e-06 MHz
           ibeam :                               -1
      machine_id :                               20
          nbeams :                                1
           nbits :                               32
          nchans :                         16777216
            nifs :                                1
     rawdatafile :                        Synthetic
     source_name :                        Synthetic
         src_dej :                     -28:22:59.16
         src_raj :                         17:47:15
    telescope_id :                                6
           tsamp :                     18.253611008
   tstart (ISOT) :          2025-06-26T20:45:59.632
    tstart (MJD) :               60852.865273

In [12]:
### Now we'll start injecting signals.

### Define here the frequencies at which you want to inject signals.
### I put them 32 kHz apart over the full frequency range, which results in 1465 signals.
### You can put them somewhat closer together than this, but if you cram too many signals into
### the data, it biases the noise calculations of turboSETI and BLISS.
### Leave at least 8 kHz of space between, as a general rule.
center_frequencies = np.arange(6048.339845146983+0.008, 6095.214842353015-0.008, 0.032)

n_inj = len(center_frequencies)
print(n_inj)

### Shuffle around the center frequencies by up to +/-50 Hz; this step isn't necessary with fully synthetic data,
### but in RFI-heavy data it helps you ensure that your injections don't line up with RFI combs with the same frequency intervals.
### (Unlikely but possible.)
cf_offsets = np.random.uniform(-0.00005, 0.00005, n_inj)
center_frequencies = center_frequencies + cf_offsets

fdiffs = np.diff(np.sort(center_frequencies))

### Define range of drift rates (in Hz/s).
drift_rates = np.random.uniform(-1, 1, n_inj)

### Define range of SNRs.
snrs = np.random.uniform(1, 1000, n_inj)

### Define signal widths (all unresolved signals in this case).
widths = np.ones_like(snrs)

print(np.min(fdiffs)*1000000)
print(len(center_frequencies))

1465
31902.206226732233
1465


In [14]:
### In this cell, we'll do the injections using the parameters we defined in the last cell.

freqs, _ = fb.grab_data()

h5_path = '/datax/scratch/jaym/synthetic_chi2_pfb_data_16ch.h5'
fb = bl.Waterfall('/datax/scratch/jaym/synthetic_chi2_pfb_data_16ch.h5')
fb.info()

n_coarse = 16
pfb_corr = np.concatenate([np.fromfile('/mnt/blpc2/datax/scratch/benjb/bliss_LSCX_test/bliss_output/GBT_spliced_PFB_response.f32', dtype='float32') for i in range(n_coarse)])
c = stg.Frame(fb)

intensity = c.get_intensity(snr=1000)

### Loop over all the frequencies — each frequency is a signal.

for i in range(len(center_frequencies)):
    if i%1 == 0:
        print(f'{i} of {len(center_frequencies)}')

    # Retrieve parameters for this signal.
    f = center_frequencies[i]
    d = drift_rates[i]
    w = widths[i]
    s = snrs[i]
    dsign = np.sign(d)

    # Define a bounding box for this signal.
    # The mins and maxes are to account for fch1 sometimes being the leftmost and sometimes the rightmost frequency.

    lb = np.min(((f-dsign*0.0001), (f+d*(1e-6)*300+dsign*0.0001)))
    rb = np.max(((f-dsign*0.0001), (f+d*(1e-6)*300+dsign*0.0001)))

    ### Noise statistics should be calculated solely from the window of injection, not the whole coarse channel.
    data_window = bl.Waterfall(h5_path, lb, rb)
    subc = stg.Frame(data_window)
    # noise_floor = np.mean(data_window)
    # intensity_diff = noise_mean - noise_floor
    # print(noise_mean)
    # print(noise_floor)
    # print(intensity_diff)
    # print(c.get_intensity(snr=1000))
    #print(f'lb: {lb} rb: {rb}')

    c.add_signal(path = stg.constant_path(f_start=(f*u.MHz),
                               drift_rate=d*u.Hz/u.s),
                           t_profile = stg.constant_t_profile(level=subc.get_intensity(snr=s)),
                           f_profile = stg.box_f_profile(width=w*c.df*u.Hz),
                           bp_profile = pfb_corr[np.argmin(np.abs(freqs-rb)):np.argmin(np.abs(freqs-lb))],
                           #stg.constant_bp_profile(level=1),
                           #integrate_t_profile=True,
                           #integrate_f_profile=True,
                           #integrate_path=True,
                           doppler_smearing=True,
                           smearing_subsamples=15,
                           bounding_f_range=(lb*u.MHz, rb*u.MHz))
    
# Convert the Frame to a numpy array and then to a Waterfall object.
# This lets us stipulate the dtype of the array, which keeps seticore from complaining.

fb_new = bl.Waterfall(filename=None, header_dict=fb.header, data_array=np.expand_dims(np.flip(c.data, axis=1), axis=1).astype('<f4'))

fb_new.write_to_hdf5('/datax/scratch/jaym/synthetic_unresolved_vardrift1_32kHz_snr1_1000.h5')


--- File Info ---
DIMENSION_LABELS :   ['time' 'feed_id' 'frequency']
        az_start :                              0.0
       data_type :                                1
            fch1 :            6095.214842353015 MHz
            foff :      -2.7939677238464355e-06 MHz
           ibeam :                               -1
      machine_id :                               20
          nbeams :                                1
           nbits :                               32
          nchans :                         16777216
            nifs :                                1
     rawdatafile :                        Synthetic
     source_name :                        Synthetic
         src_dej :                     -28:22:59.16
         src_raj :                         17:47:15
    telescope_id :                                6
           tsamp :                     18.253611008
   tstart (ISOT) :          2025-06-26T20:45:59.632
    tstart (MJD) :               60852.865273

In [16]:
### Don't forget to save the parameters you generated!

np.save('/datax/scratch/jaym/synthetic_unresolved_vardrift1_32kHz_snr1_1000_injections.npy', np.array([center_frequencies, drift_rates, snrs, widths]))